Args for training

In [3]:
args = Dict(
    "dataset" => "../../datasets/gae-benchmarks/cora.npz",
    "blurring" => 0.0,
    "feature-distribution" => Bernoulli,
    "diml1" => 32,
    "dimxiadj" => 16,
    "dimxifeat" => 16,
    "overlap" => 16,
    "nepochs" => 200,
    "savehistory" => "/dev/null",
    "saveweights" => nothing,
    "profile" => nothing
)

Dict{String,Any} with 11 entries:
  "feature-distribution" => Bernoulli
  "diml1"                => 32
  "nepochs"              => 200
  "overlap"              => 16
  "blurring"             => 0.0
  "dimxiadj"             => 16
  "dataset"              => "../../datasets/gae-benchmarks/cora.npz"
  "profile"              => nothing
  "saveweights"          => nothing
  "savehistory"          => "/dev/null"
  "dimxifeat"            => 16

Implementation of the model

In [6]:
include("../../julia/utils.jl")
include("../../julia/layers.jl")
include("../../julia/generative.jl")
include("../../julia/dataset.jl")
using .Utils
using .Layers
using .Generative
using .Dataset

using Flux
using LightGraphs
using ProgressMeter
using Statistics
using Distributions
using Random
import BSON
using ArgParse
using Profile
import JLD
using NPZ
using PyCall


# Parameters
const klscale = 1f-3
const regscale = 1f-3
const profile_losses_filename = "an2vec-losses.jlprof"
const supported_feature_distributions = [Bernoulli, Categorical, Normal]
const feature_distributions_dict = Dict(lowercase(repr(d)) => d for d in supported_feature_distributions)


"""Load the adjacency matrix and features."""
function dataset(args)
    adjfeatures = npzread(args["dataset"])

    features = transpose(adjfeatures["features"])

    # Make sure we have a non-weighted graph
    @assert Set(adjfeatures["adjdata"]) == Set([1])

    # Remove any diagonal elements in the matrix
    rows = adjfeatures["adjrow"]
    cols = adjfeatures["adjcol"]
    nondiagindices = findall(rows .!= cols)
    rows = rows[nondiagindices]
    cols = cols[nondiagindices]
    # Make sure indices start at 0
    @assert minimum(rows) == minimum(cols) == 0

    # Construct the graph
    edges = LightGraphs.SimpleEdge.(1 .+ rows, 1 .+ cols)
    g = SimpleGraphFromIterator(edges)

    # Check sizes for sanity
    @assert size(g, 1) == size(g, 2) == size(features, 2)
    g, convert(Array{Float32}, features), convert(Array{Float32}, scale_center(features))
end


"""Make the model."""
function make_vae(;g, feature_size, args)
    diml1, dimξadj, dimξfeat, overlap = args["diml1"], args["dimxiadj"], args["dimxifeat"], args["overlap"]

    # Encoder
    l1 = Layers.GC(g, feature_size, diml1, Flux.relu, initb = Layers.nobias)
    lμ = Layers.Apply(Layers.VOverlap(overlap),
        Layers.GC(g, diml1, dimξadj, initb = Layers.nobias),
        Layers.GC(g, diml1, dimξfeat, initb = Layers.nobias))
    llogσ = Layers.Apply(Layers.VOverlap(overlap),
        Layers.GC(g, diml1, dimξadj, initb = Layers.nobias),
        Layers.GC(g, diml1, dimξfeat, initb = Layers.nobias))
    enc(x) = (h = l1(x); (lμ(h), llogσ(h)))
    encparams = Flux.params(l1, lμ, llogσ)

    # Sampler
    sampleξ(μ, logσ) = μ .+ exp.(logσ) .* randn_like(μ)

    # Decoder
    decadj = Layers.Bilin()
    decfeat, decparams = if args["feature-distribution"] == Normal
        println("Info: using Gaussian feature decoder")
        decfeatl1 = Dense(dimξfeat, diml1, Flux.relu, initb = Layers.nobias)
        decfeatlμ = Dense(diml1, feature_size, initb = Layers.nobias)
        decfeatllogσ = Dense(diml1, feature_size, initb = Layers.nobias)
        decfeat(ξ) = (h = decfeatl1(ξ); (decfeatlμ(h), decfeatllogσ(h)))
        decfeat, Flux.params(decadj, decfeatl1, decfeatlμ, decfeatllogσ)
    else
        println("Info: using non-Gaussian feature decoder")
        decfeat = Chain(
            Dense(dimξfeat, diml1, Flux.relu, initb = Layers.nobias),
            Dense(diml1, feature_size, initb = Layers.nobias),
        )
        decfeat, Flux.params(decadj, decfeat)
    end
    dec(ξ) = (decadj(ξ[1:dimξadj, :]), decfeat(ξ[end-dimξfeat+1:end, :]))

    enc, sampleξ, dec, encparams, decparams
end


"""Define the function compting AUC and AP scores for model predictions (adjacency only)"""
function make_perf_scorer(;enc, sampleξ, dec, greal::SimpleGraph, test_true_edges, test_false_edges)
    # Convert test edge arrays to indices
    test_true_indices = CartesianIndex.(test_true_edges[:, 1], test_true_edges[:, 2])
    test_false_indices = CartesianIndex.(test_false_edges[:, 1], test_false_edges[:, 2])

    # Prepare ground truth values for test edges
    Areal = Array(adjacency_matrix(greal))
    real_true = Areal[test_true_indices]
    @assert real_true == ones(length(test_true_indices))
    real_false = Areal[test_false_indices]
    @assert real_false == zeros(length(test_false_indices))
    real_all = vcat(real_true, real_false)

    metrics = pyimport("sklearn.metrics")

    function perf(x)
        μ = enc(x)[1]
        Alogitpred = dec(μ)[1].data
        pred_true = Utils.threadedσ(Alogitpred[test_true_indices])
        pred_false = Utils.threadedσ(Alogitpred[test_false_indices])
        pred_all = vcat(pred_true, pred_false)

        metrics[:roc_auc_score](real_all, pred_all), metrics[:average_precision_score](real_all, pred_all)
    end

    perf
end


"""Define the model losses."""
function make_losses(;g, labels, feature_size, args, enc, sampleξ, dec, paramsenc, paramsdec)
    feature_distribution = args["feature-distribution"]
    dimξadj, dimξfeat, overlap = args["dimxiadj"], args["dimxifeat"], args["overlap"]
    Adiag = Array{Float32}(adjacency_matrix_diag(g))
    densityA = Float32(mean(adjacency_matrix(g)))
    densitylabels = Float32(mean(labels))

    # TODO check normalisation constants

    # Kullback-Leibler divergence
    Lkl(μ, logσ) = sum(Utils.threadedklnormal(μ, logσ))
    κkl = Float32(size(g, 1) * (dimξadj - overlap + dimξfeat))

    # Adjacency loss
    Ladj(logitApred) = (
        sum(threadedlogitbinarycrossentropy(logitApred, Adiag, pos_weight = (1f0 / densityA) - 1))
        / (2 * (1 - densityA))
    )
    κadj = Float32(size(g, 1)^2 * log(2))

    # Features loss
    Lfeat(logitFpred, ::Type{Bernoulli}) = (
        sum(threadedlogitbinarycrossentropy(logitFpred, labels, pos_weight = (1f0 / densitylabels) - 1))
        / (1 - densitylabels)
    )
    κfeat_bernoulli = Float32(prod(size(labels)) * log(2))
    κfeat(::Type{Bernoulli}) = κfeat_bernoulli

    Lfeat(unormFpred, ::Type{Categorical}) = - softmaxcategoricallogprob(unormFpred, labels)
    κfeat_categorical = Float32(size(g, 1) * log(feature_size))
    κfeat(::Type{Categorical}) = κfeat_categorical

    Lfeat(Fpreds, ::Type{Normal}) = ((μ, logσ) = Fpreds; sum(Utils.threadednormallogprobloss(μ, logσ, labels)))
    κfeat_normal = Float32(prod(size(labels)) * (log(2π) + mean(labels.^2)) / 2)
    κfeat(::Type{Normal}) = κfeat_normal

    # Total loss
    function losses(x)
        μ, logσ = enc(x)
        logitApred, unormfeatpred = dec(sampleξ(μ, logσ))
        Dict("kl" => klscale * Lkl(μ, logσ) / κkl,
            "adj" => Ladj(logitApred) / κadj,
            "feat" => Lfeat(unormfeatpred, feature_distribution) / κfeat(feature_distribution),
            "reg" => regscale * Utils.regularizer(paramsdec))
    end

    function loss(x)
        sum(values(losses(x)))
    end

    losses, loss
end


"""Profile runs of a function"""
function profile_fn(n::Int64, fn, args...)
    for i = 1:n
        fn(args...)
    end
end


"""Train a VAE."""
function train_vae!(;args, features, losses, loss, perf, paramsvae)
    nepochs = args["nepochs"]

    history = Dict(name => zeros(nepochs) for name in keys(losses(features)))
    history["total loss"] = zeros(nepochs)
    #history["auc"] = zeros(nepochs)
    #history["ap"] = zeros(nepochs)

    opt = ADAM(0.01)
    @showprogress for i = 1:nepochs
        Flux.train!(loss, paramsvae, [(features,)], opt)

        lossparts = losses(features)
        for (name, value) in lossparts
            history[name][i] = value.data
        end
        history["total loss"][i] = sum(values(lossparts)).data
        #history["auc"][i], history["ap"][i] = perf(features)
    end

    history
end

train_vae!

Train the model

In [7]:
println("Loading the dataset")
g, labels, features = dataset(args)
gtrain, test_true_edges, test_false_edges = Dataset.make_blurred_test_set(g, args["blurring"])
feature_size = size(features, 1)

println("Making the model")
enc, sampleξ, dec, paramsenc, paramsdec = make_vae(
    g = gtrain, feature_size = feature_size, args = args)
losses, loss = make_losses(
    g = gtrain, labels = labels, feature_size = feature_size, args = args,
    enc = enc, sampleξ = sampleξ, dec = dec,
    paramsenc = paramsenc, paramsdec = paramsdec)
perf = make_perf_scorer(
    enc = enc, sampleξ = sampleξ, dec = dec,
    greal = g, test_true_edges = test_true_edges, test_false_edges = test_false_edges)

println("Training...")
paramsvae = Flux.Params()
push!(paramsvae, paramsenc..., paramsdec...)
history = train_vae!(
    args = args, features = features,
    losses = losses, loss = loss, perf = nothing,
    paramsvae = paramsvae)

println("Final losses and performance metrics:")
for (name, values) in history
    println("  $name = $(values[end])")
end

Loading the dataset
Making the model
Info: using non-Gaussian feature decoder
Training...


Progress: 100%|████████████████████████████████████████▊|  ETA: 0:00:02

Final losses and performance metrics:
  adj = 0.6321921944618225
  feat = 0.8452295064926147
  kl = 0.004350213799625635
  reg = 0.061194006353616714
  total loss = 1.5429660081863403


Progress: 100%|█████████████████████████████████████████| Time: 0:05:11


Save the embeddings

In [11]:
embμ = enc(features)[1].data
npzwrite("cora.emb.npz", embμ')